In [1]:
import numpy as np
from gtoc13 import YPTU, SPTU, KMPDU, bodies_data
from w_utils import min_dv_lam, plot_porkchop
from itertools import product
from IPython.display import clear_output
import time

In [2]:
# Timeframe
Yo = 2
Yf = 15
gridsize = 75
dv_lim = 200 * SPTU / KMPDU  # km/s
show_plots = True

In [3]:
To = float(Yo / YPTU)
Tf = float(Yf / YPTU)
ts = np.logspace(To / 10, Tf / 10, gridsize, base=np.e)
dts = np.linspace(To + (1 / 52) / YPTU, Tf, int(np.round(2 * gridsize)))
t_dep = np.log(ts) * 5
tof = dts * 2

In [4]:
keys = [k for k, b in bodies_data.items() if b.is_planet() or b.name == "Yandi"]
keys.reverse()
KM_idxs = [(k, m) for (k, m) in product(keys, keys) if k != m]

In [ ]:
for km in KM_idxs:
    k, m = km
    dvs = [[min_dv_lam(k, ti, m, dt, debug=True) for dt in tof] for ti in t_dep]
    plot_porkchop(body1=k, body2=m, t_dep=t_dep, tof=tof, dvs=dvs, dv_limit=dv_lim, show=show_plots)
    if show_plots:
        time.sleep(5)
        clear_output(wait=True)